In [1]:
import os, sys, re, argparse, pdb, glob, pprint, json
import TS_pipe as tsp
import pandas as pd
f***REMOVED*** rdkit import Chem
f***REMOVED*** rdkit.Chem import rdmolfiles
f***REMOVED*** openbabel import pybel
pd.options.display.max_rows = 999
pd.set_option('max_colwidth', 200)
json_file = '../results/adaes_closed.json'
df = pd.read_json(json_file, orient='index')
df = df.reset_index(names='Molecule')

In [2]:
#df = df[df['TS guess key value'] < 160.0]
#df = df.reset_index()
df

,Molecule,TS mode,key parameter atoms,TS guess key parameter,TS guess key value,pysis input conformers number,pysis output conformers,pysis output conformers number,pysis conformers properties,orca input conformers number,orca output conformers,orca output conformers number,orca conformers properties
0,C18H11F6NO2S2,C1=CCNC=C1,"[7, 8, 18, 4, 5, 6]",distances N-C,2.193513,2,"[C18H11F6NO2S2/8_C18H11F6NO2S2_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz, C18H11F6NO2S2/8_C18H11F6NO2S2_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz]",2,"{'C18H11F6NO2S2/8_C18H11F6NO2S2_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz': {'TS conformer key value': 2.124968979001573}, 'C18H11F6NO2S2/8_C18H11F6NO2S2_filtered_TS_irc/cregened...",2,"[C18H11F6NO2S2/9_C18H11F6NO2S2_orca/cregened_conformers_0_reopt, C18H11F6NO2S2/9_C18H11F6NO2S2_orca/cregened_conformers_1_reopt]",2,"{'C18H11F6NO2S2/9_C18H11F6NO2S2_orca/cregened_conformers_0_reopt': {'orca TS Gibbs energy': -317.28919676, 'orca TS E corr': 0.22084733, 'orca TS def2-tzvp full energy': -2294.165945613194, 'orca ..."
1,C18H11F6NOS,C1=CCNC=C1,"[7, 8, 16, 4, 5, 6]",distances N-C,2.044234,2,"[C18H11F6NOS/8_C18H11F6NOS_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz, C18H11F6NOS/8_C18H11F6NOS_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz]",2,"{'C18H11F6NOS/8_C18H11F6NOS_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz': {'TS conformer key value': 2.024879763610135}, 'C18H11F6NOS/8_C18H11F6NOS_filtered_TS_irc/cregened_conform...",2,"[C18H11F6NOS/9_C18H11F6NOS_orca/cregened_conformers_0_reopt, C18H11F6NOS/9_C18H11F6NOS_orca/cregened_conformers_1_reopt]",2,"{'C18H11F6NOS/9_C18H11F6NOS_orca/cregened_conformers_0_reopt': {'orca TS Gibbs energy': -291.07401136, 'orca TS E corr': 0.21686822, 'orca TS def2-tzvp full energy': -1820.707793155776, 'orca reac..."
2,C18H11F6NS2,C1=CCNC=C1,"[7, 8, 16, 4, 5, 6]",distances N-C,2.040903,2,"[C18H11F6NS2/8_C18H11F6NS2_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz, C18H11F6NS2/8_C18H11F6NS2_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz]",2,"{'C18H11F6NS2/8_C18H11F6NS2_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz': {'TS conformer key value': 1.983027320471768}, 'C18H11F6NS2/8_C18H11F6NS2_filtered_TS_irc/cregened_conform...",2,"[C18H11F6NS2/9_C18H11F6NS2_orca/cregened_conformers_0_reopt, C18H11F6NS2/9_C18H11F6NS2_orca/cregened_conformers_1_reopt]",2,"{'C18H11F6NS2/9_C18H11F6NS2_orca/cregened_conformers_0_reopt': {'orca TS Gibbs energy': -285.24283527, 'orca TS E corr': 0.21323807, 'orca TS def2-tzvp full energy': -2143.667088282491, 'orca reac..."
3,C18H11F6NS2_1,C1=CCNC=C1,"[7, 8, 16, 4, 5, 6]",distances N-C,2.015901,2,"[C18H11F6NS2_1/8_C18H11F6NS2_1_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz, C18H11F6NS2_1/8_C18H11F6NS2_1_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz]",2,"{'C18H11F6NS2_1/8_C18H11F6NS2_1_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz': {'TS conformer key value': 2.014313162185135}, 'C18H11F6NS2_1/8_C18H11F6NS2_1_filtered_TS_irc/cregened...",2,"[C18H11F6NS2_1/9_C18H11F6NS2_1_orca/cregened_conformers_0_reopt, C18H11F6NS2_1/9_C18H11F6NS2_1_orca/cregened_conformers_1_reopt]",2,"{'C18H11F6NS2_1/9_C18H11F6NS2_1_orca/cregened_conformers_0_reopt': {'orca TS Gibbs energy': -285.23825433, 'orca TS E corr': 0.21299001, 'orca TS def2-tzvp full energy': -2143.661660140973, 'orca ..."
4,C19H12F6N2S2,C1=CCNC=C1,"[7, 8, 18, 4, 5, 6]",distances N-C,2.029239,2,"[C19H12F6N2S2/8_C19H12F6N2S2_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz, C19H12F6N2S2/8_C19H12F6N2S2_filtered_TS_irc/cregened_conformers_0/ts_final_geometry.xyz]",2,"{'C19H12F6N2S2/8_C19H12F6N2S2_filtered_TS_irc/cregened_conformers_1/ts_final_geometry.xyz': {'TS conformer key value': 2.027032417667819}, 'C19H12F6N2S2/8_C19H12F6N2S2_filtered_TS_irc/cregened_con...",2,"[C19H12F6N2S2/9_C19H12F6N2S2_orca/cregened_conformers_0_reopt, C19H12F6N2

In [3]:
failed_orca_count = 0
failed_orca_list = []
conformers_by_orca_count = df['pysis output conformers number'].sum()
conformers_to_orca_count = df['orca input conformers number'].sum()
energies_dict = {}
for i in df.index:
    closed_Es = []
    TS_Es = []
    for j, conf in enumerate(df.loc[i]['orca output conformers']):
        try:
            if not list(df.loc[i]['orca conformers properties'][conf].values()):
                failed_orca_count += 1
                failed_orca_list.append(conf)
            if df.loc[i]['orca conformers properties'][conf]['orca reac1 conformer key value'] > df.loc[i]['orca conformers properties'][conf]['orca reac2 conformer key value']:
                closed_energy = df.loc[i]['orca conformers properties'][conf]['orca reac2 def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca reac2 E corr']
                print('Structure with ', df.loc[i]['orca conformers properties'][conf]['orca reac2 conformer key value'], 'selected as metastable')
            else:
                closed_energy = df.loc[i]['orca conformers properties'][conf]['orca reac1 def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca reac1 E corr']
                print('Structure with ', df.loc[i]['orca conformers properties'][conf]['orca reac1 conformer key value'], 'selected as metastable')
            TS_energy =  df.loc[i]['orca conformers properties'][conf]['orca TS def2-tzvp full energy'] + df.loc[i]['orca conformers properties'][conf]['orca TS E corr']
        except KeyError as e:
            print(f'Conformer {conf} is missing')
            print(e)
            continue
        closed_Es.append(closed_energy)
        TS_Es.append(TS_energy)
    if not ( closed_Es and TS_Es ):
        continue
    energies_dict[f"{df.loc[i]['Molecule']}"] = {}
    energies_dict[f"{df.loc[i]['Molecule']}"]['Closed energies'] = closed_Es
    energies_dict[f"{df.loc[i]['Molecule']}"]['TS energies'] = TS_Es
print(f'Conformers generated by ORCA stage: {conformers_to_orca_count}')
print(f'Failed ORCA jobs: {failed_orca_count}')
print(f'Failed ORCA list:', failed_orca_list)
pprint.pprint(energies_dict)
print(len(energies_dict.keys()))

with open('adaes_closed_final_energies.json', 'w') as json_file:
    json.dump(energies_dict, json_file, indent=4)



Structure with  1.450285746748779 selected as metastable
Structure with  1.448769569703844 selected as metastable
Structure with  1.454450324543008 selected as metastable
Structure with  1.452045369819326 selected as metastable
Structure with  1.463451484398278 selected as metastable
Structure with  1.4602577787957731 selected as metastable
Structure with  1.478232973842916 selected as metastable
Structure with  1.476024575650709 selected as metastable
Structure with  1.458704814127677 selected as metastable
Structure with  1.457328775425478 selected as metastable
Structure with  1.462568322510255 selected as metastable
Structure with  1.461447234416418 selected as metastable
Structure with  1.473708095987246 selected as metastable
Structure with  1.4720945491209632 selected as metastable
Structure with  1.459126377514139 selected as metastable
Structure with  1.458412674113295 selected as metastable
Conformers generated by ORCA stage: 16
Failed ORCA jobs: 0
Failed ORCA list: []
{'C18H

In [4]:
os.getcwd()

'C:\\Users\\***REMOVED***pe672\\Documents\\GitHub\\TS_pipeline\\src'